**Bases de datos vectoriales**

En esta sección, demostramos algunos casos de uso comunes de incrustación con algunos ejemplos simples. La distancia euclidiana se utiliza para calcular la similitud entre dos fragmentos de texto.

**Búsqueda y recomendación**

Supongamos que tiene una colección de documentos (el conjunto de datos). Cada documento está representado por su incrustación. Se le ha proporcionado una cadena de consulta. La solicitud es identificar el documento que es más relevante para la cadena de consulta. Puede lograr esto con los siguientes pasos:

Se tarda unos 3 minutos en generar la incrustación de 1000 frases de forma secuencial.

In [1]:
import json
import boto3

def get_embedding(bedrock, text):
    modelId = 'amazon.titan-embed-text-v1'
    accept = 'application/json'
    contentType = 'application/json'
    input = {
            'inputText': text
        }
    body=json.dumps(input)
    response = bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept,contentType=contentType)
    response_body = json.loads(response.get('body').read())
    embedding = response_body['embedding']
    return embedding

# main function
bedrock = boto3.client(
    service_name='bedrock-runtime'
)
# some random data
people = ['Albert Einstein', 'Isaac Newton', 'Stephen Hawking', 
          'Galileo Galilei', 'Niels Bohr', 'Werner Heisenberg', 
          'Marie Curie', 'Ernest Rutherford', 'Michael Faraday', 'Richard Feynman']
actions = ['plays basketball', 'teaches physics', 'sells sea shells', 
           'collects tax', 'drives buses', 'researches into gravity', 
           'manages a shop', 'supervises graduate students', 
           'works as a support engineer', 'runs a bank']
places = ['London', 'Sydney', 'Los Angeles', 'San Francisco', 'Beijing', 
          'Cape Town', 'Paris', 'Cairo', 'New Delhi', 'Seoul']
# create a data file
count = 10000
with open('dataset.json', 'w') as outfile:
    for name in people:
        for action in actions:
            for place in places:
                id   = count
                text = '{name} {action} in {place}.'.format(name=name, action=action, place=place)
                embedding = get_embedding(bedrock, text)
                item = {'id': id, 'text': text, 'embedding': embedding}
                json_object = json.dumps(item)
                outfile.write(json_object + '\n')
                count = count + 1
    print('Dataset created.')


Dataset created.


Espere a que el código termine de ejecutarse e inspeccione el tamaño del archivo de salida. Con 1000 entradas de datos, el tamaño del archivo es de 18.767.288 bytes. En promedio, es de aproximadamente 18767 bytes por entrada de datos. Tenga en cuenta que el tamaño del id y el texto combinados es inferior a 100 bytes. La incrustación es el componente principal en términos de tamaño de datos.

En el siguiente código de ejemplo se muestra cómo realizar una búsqueda entre el conjunto de datos.

In [2]:
import json
import boto3
import math
from datetime import datetime

def get_embedding(bedrock, text):
    modelId = 'amazon.titan-embed-text-v1'
    accept = 'application/json'
    contentType = 'application/json'
    input = {
            'inputText': text
        }
    body=json.dumps(input)
    response = bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept,contentType=contentType)
    response_body = json.loads(response.get('body').read())
    embedding = response_body['embedding']
    return embedding

def load_dataset(filename):
    dataset = []
    with open(filename) as file:
        for line in file:
            dataset.append(json.loads(line))
    return dataset

def calculate_distance(v1, v2):
    distance = math.dist(v1, v2)
    return distance
    
def search(dataset, embedding):
    t1 = datetime.now()
    for item in dataset:
        item['distance'] = calculate_distance(item['embedding'], embedding)
    t2 = datetime.now()
    delta = t2 - t1
    ms1 = 1000 * delta.total_seconds()
    dataset.sort(key=lambda x: x['distance'])
    t3 = datetime.now()
    delta = t3 - t2
    ms2 = 1000 * delta.total_seconds()
    print(str(ms1) + 'ms in calculating distances')
    print(str(ms2) + 'ms in sorting distances')
    return dataset[0]['text']

# main function
bedrock = boto3.client(
    service_name='bedrock-runtime'
)
dataset = load_dataset('dataset.json')
query   = 'Lady Gaga purchased a necklace in Singapore.'
embedding = get_embedding(bedrock, query)
result  = search(dataset, embedding)
print(result)


40.870999999999995ms in calculating distances
0.294ms in sorting distances
Marie Curie sells sea shells in Los Angeles.


Este código de ejemplo también imprime el tiempo que dedicamos a calcular las distancias (que es un componente principal) y el tiempo que dedicamos a ordenar las distancias (que es un componente secundario).

Tenga en cuenta que calculamos la distancia entre el texto de la consulta y todas las entradas del conjunto de datos. Con este diseño, la complejidad de cálculo es cercana a O(n). Al agregar más datos al conjunto de datos, se espera que el tiempo de cálculo aumente linealmente.

**pgvector**

PostgreSQL tiene una extensión pgvector para la búsqueda de similitud vectorial. La ventaja de pgvector es que la mayoría de los lenguajes de programación tienen la biblioteca para conectarse a PostgreSQL. Además, los clientes pueden usar su cliente SQL favorito para trabajar con los datos durante la creación de prototipos y la resolución de problemas.

Amazon RDS for PostgreSQL (versión 15.3 o posterior) es compatible con pgvector. Para este taller, se ha creado una instancia de RDS PosgreSQL en su cuenta de AWS en el paso Prerequisites (Requisitos previos). Las credenciales de conexión a la base de datos se almacenan en AWS Secrets Manager. En la consola de AWS Secrets Manager, haga clic en Secrets (Secretos) en el panel de navegación para ver una lista de los secretos de su cuenta de AWS. El nombre del secreto de este taller es bedrock-workshop-xxxxxxxx. Vamos a almacenar el nombre secreto en una variable para que podamos usarlo más tarde.

In [4]:
secret_name = 'bedrock-workshop-78a084d0'

**Cargar datos de muestra**

El siguiente código de ejemplo carga el conjunto de datos que creamos anteriormente en la base de datos.

In [5]:
import json
import boto3
import psycopg2
from botocore.exceptions import ClientError

def get_secrets():
    client = boto3.client(
        service_name='secretsmanager',
    )
    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e
    secrets = json.loads(get_secret_value_response['SecretString'])
    return secrets
    
def load_dataset(filename):
    dataset = []
    with open(filename) as file:
        for line in file:
            dataset.append(json.loads(line))
    return dataset
    
# main function
secrets = get_secrets()
conn = psycopg2.connect(
    host=secrets['db_hostname'],
    port=secrets['db_hostport'],
    user=secrets['db_username'],
    password=secrets['db_password'],
    database=secrets['db_database']
)
cursor = conn.cursor()
cursor.execute('CREATE EXTENSION vector')
cursor.execute('CREATE TABLE dataset (id SERIAL, content TEXT, embedding VECTOR(1536))')
conn.commit()
print('Table created.')


Table created.


**Realizar una búsqueda**

En el siguiente código de ejemplo se muestra cómo realizar una búsqueda entre el conjunto de datos.

En la instrucción SQL, operador significa usar la distancia euclidiana para calcular la similitud entre vectores. También puede utilizar el operador de producto interno o la distancia de coseno para calcular la similitud entre vectores.<-><#><=>

In [6]:
import json
import boto3
import psycopg2
from botocore.exceptions import ClientError
from datetime import datetime

def get_secrets():
    client = boto3.client(
        service_name='secretsmanager',
    )
    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e
    secrets = json.loads(get_secret_value_response['SecretString'])
    return secrets

def get_embedding(bedrock, text):
    modelId = 'amazon.titan-embed-text-v1'
    accept = 'application/json'
    contentType = 'application/json'
    input = {
            'inputText': text
        }
    body=json.dumps(input)
    response = bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept,contentType=contentType)
    response_body = json.loads(response.get('body').read())
    embedding = response_body['embedding']
    return embedding

def search(bedrock, cursor, query, limit):
    embedding = str(get_embedding(bedrock, query))
    sql = 'SELECT id, content FROM dataset ORDER BY embedding <-> %s LIMIT %s'
    cursor.execute(sql, (embedding, limit))
    result = []
    for row in cursor:
        result.append(row)
    return result
    
# main function
bedrock = boto3.client(
    service_name='bedrock-runtime'
)
secrets = get_secrets()
conn = psycopg2.connect(
    host=secrets['db_hostname'],
    port=secrets['db_hostport'],
    user=secrets['db_username'],
    password=secrets['db_password'],
    database=secrets['db_database']
)
cursor = conn.cursor()
query   = 'Lady Gaga purchased a necklace in Singapore.'
result = search(bedrock, cursor, query, 1)
print(result)


[]


En comparación con el enfoque de archivos JSON, pgvector facilita las cosas al ocultar los detalles relacionados con el almacenamiento y el algoritmo. Todo lo que necesita hacer es enviar una consulta SQL al servidor de base de datos.

**Búsqueda vectorial sin servidor de OpenSearch**

Amazon OpenSearch Serverless es una configuración sin servidor bajo demanda para Amazon OpenSearch Service. La tecnología sin servidor elimina las complejidades operativas del aprovisionamiento, la configuración y el ajuste de los clústeres de OpenSearch. Una colección sin servidor de OpenSearch es un grupo de índices de OpenSearch que funcionan juntos para admitir una carga de trabajo o un caso de uso específicos. Las colecciones son más fáciles de usar que los clústeres de OpenSearch autoadministrados, que requieren aprovisionamiento manual.

Para este taller, se ha creado una colección OpenSearch Serverless en su cuenta de AWS en el paso Prerequisites (Requisitos previos). En la consola de Amazon OpenSearch Service, seleccione Collections (Colecciones) en el panel de navegación. El nombre de la colección de este taller es bedrock-workshop-collection. Haga clic en el nombre de la colección para ver los detalles de la colección. Tome nota del punto de conexión (host) de OpenSearch de la colección.

Vamos a crear dos variables para almacenar el nombre de la región de AWS y el punto de enlace de OpenSearch (host) de la colección OpenSearch Serverless. En este ejemplo, usamos la región us-west-2. Debe cambiarlo a la región de AWS que está utilizando.

In [7]:
region = 'us-west-2'
host = 'https://5tl3o6l27r1pcm2hddk9.us-west-2.aoss.amazonaws.com'

**Crear un índice**

En el código de ejemplo siguiente se crea un nuevo índice en la colección. El nombre del índice es demo-index. Asegúrese de utilizar la región y el host de AWS correctos (punto de enlace de OpenSearch) en el código.

In [8]:
import boto3
import requests
from requests_aws4auth import AWS4Auth

service = 'aoss'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(
    credentials.access_key, 
    credentials.secret_key, 
    region, 
    service, 
    session_token=credentials.token
)

index = 'demo-index'
url = host + '/' + index

headers = {'Content-Type': 'application/json'}
document = {
   'settings': {
      'index.knn': True
   },
   'mappings': {
      'properties': {
         'embedding': {
            'type': 'knn_vector',
            'dimension': 1536
         },
         'content': {
            'type': 'text'
         }
      }
   }
}
response = requests.put(url, auth=awsauth, json=document, headers=headers)
response.raise_for_status()
print(response.json())

#Si se produce el siguiente error, debe volver a comprobar los permisos de este rol de IAM AmazonBedrockWorkshopStackSageMakerRole y
#la política de acceso a datos para la recopilación sin servidor de OpenSearch.
#HTTPError: 403 Client Error: Forbidden for url: https://********.********.amazonaws.com/demo-index


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'demo-index'}


In [9]:
import json
import boto3
import requests
from requests_aws4auth import AWS4Auth

def load_dataset(filename):
    dataset = []
    with open(filename) as file:
        for line in file:
            dataset.append(json.loads(line))
    return dataset

# main function
service = 'aoss'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)

index = 'demo-index'
datatype = '_doc'
url = host + '/' + index + '/' + datatype

headers = {'Content-Type': 'application/json'}
dataset = load_dataset('dataset.json')
for item in dataset:
    document = {
        'embedding': item['embedding'],
        'content': item['text']
    }
    response = requests.post(url, auth=awsauth, json=document, headers=headers)
print('Data loaded into OpenSearch Serverless collection.')


Data loaded into OpenSearch Serverless collection.


**Realizar una búsqueda**

En el siguiente código de ejemplo se muestra cómo realizar una búsqueda entre el conjunto de datos.

In [10]:
import json
import boto3
import requests
from requests_aws4auth import AWS4Auth

def get_embedding(bedrock, text):
    modelId = 'amazon.titan-embed-text-v1'
    accept = 'application/json'
    contentType = 'application/json'
    input = {
            'inputText': text
        }
    body=json.dumps(input)
    response = bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept,contentType=contentType)
    response_body = json.loads(response.get('body').read())
    embedding = response_body['embedding']
    return embedding

def search(region, host, index, embedding, limit):
    credentials = boto3.Session().get_credentials()
    awsauth = AWS4Auth(
        credentials.access_key, 
        credentials.secret_key, 
        region, 
        "aoss", 
        session_token=credentials.token
    )
    datatype = '_search'
    url = host + '/' + index + '/' + datatype
    headers = {'Content-Type': 'application/json'}
    document = {
        'size': limit,
        'query': {
            'knn': {
                'embedding': {
                    'vector': embedding,
                    'k': limit
                }
            }
        }
    }
    response = requests.get(url, auth=awsauth, json=document, headers=headers)
    response.raise_for_status()
    return response.json()

# main function
bedrock = boto3.client(
    service_name='bedrock-runtime'
)
query = 'Lady Gaga purchased a necklace in Singapore.'
embedding = get_embedding(bedrock, query)
index = 'demo-index'
limit = 5
result = search(region, host, index, embedding, limit)

for item in result['hits']['hits']:
    print(item['_source']['content'])


Marie Curie sells sea shells in Los Angeles.
Marie Curie manages a shop in Los Angeles.
Marie Curie sells sea shells in Sydney.
Marie Curie sells sea shells in Seoul.
Marie Curie sells sea shells in Beijing.
